In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import time
import os

import eikon as ek
ek.set_app_key('341ab68907a740f38e7b07084d4055e3982b5bdf')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50) 
pd.set_option('display.max_columns', 50) 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [15]:
df = pd.read_excel(r"W:\019_Glassdoor\1 Data\1 Glassdoor Links\2 Eikon Sample\111622_4562_deals.xlsx")

# set permid from float to int
for permid_column in [col for col in df.columns if 'permid' in col or "code" in col]:
    df[permid_column] = df[permid_column].astype("Int64")

df[:3]
df.shape

,eikon_deal_id,date_announced,date_effective,acquiror,deal_value,shares_6_months_prior_announcement,shares_owned_after_transaction,acquiror_date_of_financials,acquiror_isin,acquiror_permid,acquiror_nation,acquiror_trbc_industry_group_code,acquiror_trbc_industry_group_name,acquiror_total_assets,acquiror_net_income,acquiror_multiple_deal_involvment,acquiror_full_name,acquiror_full_name_legal_addon,acquiror_full_name_cleaned,acquiror_best_match_name,acquiror_best_match_review_amount,acquiror_best_match_cosine,acquiror_best_match_link,acquiror_best_match_type,acquiror_link_hannes,...,target_multiple_deal_involvment,target_full_name,target_full_name_legal_addon,target_full_name_cleaned,target_best_match_name,target_best_match_review_amount,target_best_match_cosine,target_best_match_link,target_best_match_type,target_link_hannes,target_link_hannes_nonstrict,target_link_marius,target_firm_hannes,target_firm_hannes_nonstrict,target_firm_marius,target_review_amount_hannes,target_review_amount_hannes_nonstrict,target_review_amount_marius,target_link_found_hannes,target_link_found_hannes_nonstrict,target_similarity_hannes,target_similarity_hannes_nonstrict,target_similarity_marius,target_similarity_mean,target_marius_equalto_hannes
0,34391733382,2008-01-07,2008-03-04,1,2.829622e+08,0.0,100.0,2007-12-31,CA67072Q1046,4295861226,Canada,501020,Oil & Gas,6.856333e+08,2.642212e+07,0,NuVista Energy Ltd,Ltd,NuVista Energy,NuVista Energy,8.0,1.00,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,hannes,https://www.glassdoor.com/Reviews/NuVista-Energy-Reviews-E23169.htm,...,0,Rider Resources Ltd,Ltd,Rider Resources,Rider Resources,NaN,1.0,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,hannes,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,https://www.glassdoor.com/Reviews/Rider-Resources-Reviews-E23380.htm,Rider Resources,Rider Resources,Rider Resources,NaN,NaN,1.0,transformed overview link,transformed overview link,1.0,1.0,1.0,1.000000,1
1,34391733368,2008-01-07,2008-03-05,1,4.397050e+07,0.0,100.0,2007-12-31,US4519061018,4295904213,United States,521020,"Machinery, Tools, Heavy Vehicles, Trains & Ships",1.408352e+09,1.766600e+07,1,L-1 Identity Solutions Inc,Inc,L-1 Identity Solutions,Identity Solutions,NaN,0.83,https://www.glassdoor.com/Reviews/Identity-Solutions-Reviews-E4835029.htm,nonstrict,NaN,...,0,Bioscrypt Inc,Inc,Bioscrypt,Bioscrypt,2.0,1.0,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,hannes,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,https://www.glassdoor.com/Reviews/Bioscrypt-Reviews-E24042.htm,https://www.glassdoor.ca/Reviews/Bioscrypt-Reviews-E24042.htm,Bioscrypt,Bioscrypt,Bioscrypt,2.0,2.0,2.0,transformed overview link,transformed overview link,1.0,1.0,1.0,1.000000,1
2,34391297298,2008-01-08,2008-07-18,1,5.716800e+07,0.0,100.0,2007-12-31,US62845B1044,4295914259,United States,551010,Banking Services,9.573430e+08,4.226000e+06,0,MutualFirst Financial Inc,Inc,MutualFirst Financial,Mutual of Omaha,805.0,0.36,https://www.glassdoor.com/Reviews/Mutual-of-Omaha-Reviews-E3678.htm,nonstrict,NaN,...,0,MFB Corp,Corp,MFB,MFB,3.0,1.0,https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,nonstrict,"https://www.glassdoor.com/job-listing/occupational-therapist-mary-free-bed-rehabilitation-hospital-JV_IC1134796_KO0,22_KE23,60.htm?jl=1008181286734",https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,https://www.glassdoor.com/Reviews/MFB-Reviews-E2628.htm,"occupational therapist mary free bed rehabilitation hospital JV_IC1134796_KO0,22_KE23,60.htm?jl=1008181286734",MFB,MFB,NaN,3.0,3.0,wrong link,transformed overview link,0.0,1.0,1.0,0.666667,1


(4562, 73)

In [18]:
df_sic = df[df["acquiror_trbc_industry_group_code"].isna() == True]
df_sic

,eikon_deal_id,date_announced,date_effective,acquiror,deal_value,shares_6_months_prior_announcement,shares_owned_after_transaction,acquiror_date_of_financials,acquiror_isin,acquiror_permid,acquiror_nation,acquiror_trbc_industry_group_code,acquiror_trbc_industry_group_name,acquiror_total_assets,acquiror_net_income,acquiror_multiple_deal_involvment,acquiror_full_name,acquiror_full_name_legal_addon,acquiror_full_name_cleaned,acquiror_best_match_name,acquiror_best_match_review_amount,acquiror_best_match_cosine,acquiror_best_match_link,acquiror_best_match_type,acquiror_link_hannes,...,target_multiple_deal_involvment,target_full_name,target_full_name_legal_addon,target_full_name_cleaned,target_best_match_name,target_best_match_review_amount,target_best_match_cosine,target_best_match_link,target_best_match_type,target_link_hannes,target_link_hannes_nonstrict,target_link_marius,target_firm_hannes,target_firm_hannes_nonstrict,target_firm_marius,target_review_amount_hannes,target_review_amount_hannes_nonstrict,target_review_amount_marius,target_link_found_hannes,target_link_found_hannes_nonstrict,target_similarity_hannes,target_similarity_hannes_nonstrict,target_similarity_marius,target_similarity_mean,target_marius_equalto_hannes
2414,34391736686,2008-02-18,2008-06-11,1,1.055081e+08,0.0,100.0,2008-02-18,GG00BM8B8T38,4295894726,United Kingdom,<NA>,NaN,NaN,NaN,0,Bear Stearns Private Equity Ltd,Ltd,Bear Stearns Private Equity,Bear Stearns,89.0,0.65,https://www.glassdoor.com/Reviews/Bear-Stearns-Reviews-E85.htm,marius,NaN,...,0,Macquarie Pvt Capital Group,NaN,Macquarie Pvt Capital Group,Macquarie Group,61.0,0.52,https://www.glassdoor.com/Reviews/Macquarie-Group-Reviews-E12088.htm,marius,NaN,https://www.glassdoor.com/Reviews/Macquarie-Group-Reviews-E12088.htm,https://www.glassdoor.com/Reviews/Macquarie-Group-Reviews-E12088.htm,NaN,Macquarie Group,Macquarie Group,NaN,309.0,61.0,no link found,transformed overview link,0.00,0.50,0.52,0.340000,1
2424,34391313645,2009-08-05,2010-02-19,1,1.330830e+08,0.0,100.0,2009-12-31,NaN,4295899948,United States,<NA>,NaN,4.023417e+10,6.520448e+09,1,Google Inc,Inc,Google,Google,26100.0,1.00,https://www.glassdoor.com/Reviews/Google-Reviews-E9079.htm,nonstrict,https://www.glassdoor.com/Reviews/Employee-Review-Google-RVW36923113.htm,...,0,On2 Technologies Inc,Inc,On2 Technologies,On2 Technologies,NaN,1.00,https://www.glassdoor.com/Reviews/On2-Technologies-Reviews-E9482.htm,hannes,https://www.glassdoor.com/Reviews/On2-Technologies-Reviews-E9482.htm,https://www.glassdoor.com/Reviews/On2-Technologies-Reviews-E9482.htm,https://www.glassdoor.co.uk/Reviews/ON2-Solutions-Reviews-E7285836.htm,On2 Technologies,On2 Technologies,ON2 Solutions,NaN,NaN,1.0,transformed overview link,transformed overview link,1.00,1.00,0.18,0.726667,0
2426,34392093822,2011-08-15,2012-05-22,1,1.244995e+10,0.0,100.0,2011-06-30,NaN,4295899948,United States,<NA>,NaN,6.486100e+10,9.014000e+09,1,Google Inc,Inc,Google,Google,26100.0,1.00,https://www.glassdoor.com/Reviews/Google-Reviews-E9079.htm,nonstrict,https://www.glassdoor.com/Reviews/Employee-Review-Google-RVW36923113.htm,...,0,Motorola Mobility Holdings Inc,Holdings Inc,Motorola Mobility,Motorola Mobility,NaN,1.00,https://www.glassdoor.com/Reviews/Motorola-Mobility-Reviews-E451.htm,nonstrict,"https://www.glassdoor.com/Benefits/Motorola-Mobility-US-Benefits-EI_IE451.0,17_IL.18,20_IN1.htm",https://www.glassdoor.com/Reviews/Motorola-Mobility-Reviews-E451.htm,NaN,Motorola Mobility US Benefits,Motorola Mobility,NaN,259.0,NaN,NaN,wrong link,transformed overview link,0.72,1.00,0.00,0.573333,0
2428,34391300689,2008-04-22,2008-06-02,1,7.155840e+08,0.0,100.0,2008-03-31,GB0009252882,4295895781,United Kingdom,<NA>,NaN,5.570449e+10,9.927826e+09,1,GlaxoSmithKline PLC,PLC,GlaxoSmithKline,"manufacturing production operator 2nd shift gsk JV_IC1152365_KO0,43_KE44,47.htm?jl=1008175721070",NaN,0.00,"https://www.glassdoor.com/job-listing/manufacturing-production-operator-2nd-shift-gsk-JV_IC1152365_KO0,43_KE44,47.htm

## get static firm infos

In [22]:
# create target_permid list
ids = ["4295911976", "CA5658511020", "4295861595"] #first convert to int (without decimal), then to str, since get_data needs strings in the list
# ids = df_sic["acquiror_permid"].astype(str).tolist()[:10]
fields = ['TR.ISIN', 'TR.RIC', 'TR.OrganizationID', 'TR.UltimateParentId', 'TR.CompanyName', 'TR.FormerlyKnownAsName', 'TR.SucceededBy', 'TR.TRBCIndustryCode'] # check the data item browser

df_static, err = ek.get_data(ids, fields)

df_static

,Instrument,ISIN,RIC,Organization PermID,Ultimate Parent Id,Company Name,Formerly Known As Name,Organization Succeeded By,SIC Industry Code,TRBC Industry Code
0,4295911976,US92343V1044,VZ.N,4295911976,4295911976,Verizon Communications Inc,Bell Atlantic Corp,,4813,57401010
1,CA5658511020,CA5658511020,MAR.TO^L10,4295861595,4295861595,Marathon PGM Corp,,Stillwater Canada Inc,<NA>,51201010
2,4295861595,,,4295861595,4295861595,Marathon PGM Corp,,Stillwater Canada Inc,<NA>,51201010


### create list of "formerly known as"

In [ ]:
df_formerly = df_static.groupby('Instrument')['Formerly Known As Name'].apply(list).reset_index(name='other_names')
df_formerly

,Instrument,other_names
0,4295899948,[Google Inc]
1,4295904162,"[Heritage Propane Partners L P, Energy Transfe..."
2,4295911990,"[Chemex-Delaware Inc, Chemex Pharmaceuticals I..."
